## Youtube comments spam detection using auto-label

Import libraries and Setup the API keys:

In [26]:
import os
import pandas as pd
import evadb
from dataframe.labeling_agent import LabelingAgent
os.environ['OPENAI_API_KEY'] = 'sk-xxx'

Declare config and csv file:

In [27]:
config = {
    "task_name": "SpamClassification",
    "task_type": "classification",
    "dataset": {
      "label_column": "CLASS",
      "label_separator": ", ",
      "delimiter": ","
    },
    "model": {
      "provider": "openai",
      "name": "gpt-3.5-turbo"
    },
    "prompt": {
      "task_guidelines": "You are an expert at identifying spam and legitimate messages. Your goal is to maintain the quality of communication by accurately classifying incoming messages as either 'spam' or 'ham' (legitimate). Any message that is unsolicited, contains promotional content, or attempts to deceive or defraud users should be labeled as 'spam'. Messages that are personal, non-promotional, and relevant should be marked as 'ham'. Your job is to correctly label the provided input example into one of the following categories:\n{labels}",
      "output_guidelines": "You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: \"label1, label2, label3\"",
      "labels": [
        "spam",
        "ham"
      ],
      "few_shot_examples": "spam-ham-label/data/seed.csv",
      "few_shot_selection": "semantic_similarity",
      "few_shot_num": 5,
      "example_template": "Input: {example}\nOutput: {labels}\n"
    }
  }
  
csv_file = "spam-ham-label/data/test1.csv"
  

In [28]:
cursor = evadb.connect().cursor()
print("Connected to EvaDB")

Connected to EvaDB


In [29]:
create_function_query = f"""CREATE FUNCTION IF NOT EXISTS AutoLabel
            IMPL  './functions/autolabel.py';
            """
cursor.query("DROP FUNCTION IF EXISTS AutoLabel;").execute()
cursor.query(create_function_query).execute()
print("Created Function")

create_table_query = f"""
CREATE TABLE IF NOT EXISTS YTCOMMENTS(
COMMENT_ID INTEGER,
AUTHOR TEXT(30),
DATE TEXT(10),
CONTENT TEXT(255)
);
"""

load_data_query = f""" LOAD CSV 'spam-ham-label/data/test1.csv' INTO YTCOMMENTS;""" 

11-02-2023 22:19:37 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_function:0086] Function AutoLabel does not exist, therefore cannot be dropped.


Created Function


Create table and load data:

In [30]:
cursor.query(create_table_query).execute()
cursor.query(load_data_query).execute()

11-02-2023 22:19:39 WARNING[sqlite_storage_engine:sqlite_storage_engine.py:create:0108] Table {table.name} already exists


Perform Data Labeling:

In [31]:
query= f""" SELECT AutoLabel() FROM YTCOMMENTS;"""
result = cursor.query(query).execute()
print(result)

/home/preethi/projects/evadb-auto-label/dataframe/labeling_agent.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.df = df
11-02-2023 22:19:42 ERROR [plan_executor:plan_executor.py:execute_plan:0179] No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.
Traceback (most recent call last):
  File "/home/preethi/anaconda3/lib/python3.11/site-packages/evadb/executor/plan_executor.py", line 175, in execute_plan
    yield from output
  File "/home/preethi/anaconda3/lib/python3.11/site-packages/evadb/executor/project_executor.py", line 49, in

LLM PROMTTT:  You are an expert at identifying spam and legitimate messages. Your goal is to maintain the quality of communication by accurately classifying incoming messages as either 'spam' or 'ham' (legitimate). Any message that is unsolicited, contains promotional content, or attempts to deceive or defraud users should be labeled as 'spam'. Messages that are personal, non-promotional, and relevant should be marked as 'ham'. Your job is to correctly label the provided input example into one of the following categories:
spam, ham
You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"
Some examples with their output answers are provided below:
Input: z12bfraboyajftgbz04ccbkr3xjxfxyxsew, Abdullah Fawzi, 2015-05-25 6:25:22, coby this USL and past :<br /><a href="http://adf.ly">http://adf.ly</a> /1HmVtX<br />delete space after y﻿
Output: spam
Input: z132svd4fvq1wntfd221w5szfzezjri2r, Abdullah Fawzi, 2015-05-25 6:

ExecutorError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

Validate results

In [ ]:
#comapare output and validate csv